In [3]:
import subprocess
import shlex
import os
from collections import defaultdict

import pandas as pd
import numpy as np
from IPython.display import display, HTML

Number of reads:

# 1. Raw data analysis

In [ ]:
pd.DataFrame('')

In [33]:
for filename in os.listdir():
    if filename.endswith('.fastq'):
        command = f"grep '@SRR' {filename} | wc -l"
        proc = subprocess.run(command, capture_output=True, shell=True)
        print(filename, proc.stdout.decode('utf-8'))

ferm30rep1.fastq 1721675

ferm0rep2.fastq 9929568

ferm30rep2.fastq 6172452

ferm0rep1.fastq 9043877



Also run Fastq: high duplication level and adapters are present, but...

- https://academic.oup.com/nargab/article/2/3/lqaa068/5901066
- https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-0956-2

...so no filtering is required

# 2. Alignment

In [21]:
genome_index = 'yeast_index'
transcriptome_index = 'yeast_rna_index'

def align_command(index, fastq_file, p='8'):
    name = fastq_file.split('.')[0]
    if 'rna' in index:
        name += '_rna'
    command = f'hisat2 -p {str(p)} -x {index} -q {fastq_file} | samtools sort > {name}.bam'
    return shlex.split(command)

# strange thing happened:
# can't run command form jupyter with -U option: says this option is invalid
# however it runs if -U is replaced with -q
# run both commands (with -U and -q) from terminal and the results were the same, so...

for filename in os.listdir():
    if filename.endswith('.fastq'):
        for index in (genome_index, ): # transcriptome_index,):
            align = align_command(index, filename)
            print(' '.join(align))
            proc = subprocess.run(align, capture_output=True, check=True)
            print(proc.stderr.decode('utf-8'))
            print(proc.stdout.decode('utf-8'))

hisat2 -p 8 -x yeast_index -q ferm30rep1.fastq | samtools sort > ferm30rep1.bam
1721675 reads; of these:
  1721675 (100.00%) were unpaired; of these:
    66371 (3.86%) aligned 0 times
    1507906 (87.58%) aligned exactly 1 time
    147398 (8.56%) aligned >1 times
96.14% overall alignment rate


hisat2 -p 8 -x yeast_index -q ferm0rep2.fastq | samtools sort > ferm0rep2.bam
9929568 reads; of these:
  9929568 (100.00%) were unpaired; of these:
    511726 (5.15%) aligned 0 times
    8644577 (87.06%) aligned exactly 1 time
    773265 (7.79%) aligned >1 times
94.85% overall alignment rate
[bam_sort_core] merging from 2 files and 1 in-memory blocks...


hisat2 -p 8 -x yeast_index -q ferm30rep2.fastq | samtools sort > ferm30rep2.bam
6172452 reads; of these:
  6172452 (100.00%) were unpaired; of these:
    234531 (3.80%) aligned 0 times
    5367720 (86.96%) aligned exactly 1 time
    570201 (9.24%) aligned >1 times
96.20% overall alignment rate
[bam_sort_core] merging from 1 files and 1 in-memor

In [4]:
# mean genome alignment rate
s = [
    96.14,
    94.85,
    96.20,
    94.25
]
np.mean(s)

95.36

# GFF to GTF & featureCounts & DESeq2
```bash
gffread GCF_000146045.2_R64_genomic.gff -T -o GCF_000146045.2_R64_genomic.gtf
```


In [114]:
reps = [('rep1', 'rep2'), ('rep1_rna', 'rep2_rna')]
for rep in reps:
    rep1, rep2 = rep
    if 'rna' in rep1:
        continue   # it actually doesn't align...
        title = 'rna'
    else:
        title = 'gen'
        field = 'gene_id'
        # field = 'gene_id'
        # field = 'transcript_id'
        
    command = (f'featureCounts -g {field} -T 8 -a GCF_000146045.2_R64_genomic.gtf -o '
               f'{title}.fc ferm0{rep1}.bam ferm0{rep2}.bam ferm30{rep1}.bam ferm30{rep2}.bam')
    proc = subprocess.run(command, shell=True, check=True, capture_output=True)
    proc = subprocess.run(f'cat {title}.fc | cut -f 1,7-10 | R -f deseq2.r', shell=True, capture_output=True)
    print(proc.stderr.decode('utf-8'))
    subprocess.run(f'head result.txt -n 50 | cut -f 1 > {title}_genes.txt', shell=True)
    subprocess.run(f'head result.txt -n 50 > {title}_result_50.txt', shell=True)
    proc = subprocess.run(f'mv result.txt {title}_result_all.txt', shell=True, 
                          check=False, capture_output=True)
    print(proc.stderr.decode('utf-8'))
    proc = subprocess.run("cat norm-matrix-deseq2.txt | R -f draw-heatmap.r", shell=True, capture_output=True, 
                         check=True)
    subprocess.run(f'mv output.pdf {title}.pdf', shell=True)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following objec

In [ ]:
ferm30rep1.fastq 1721675

ferm0rep2.fastq 9929568

ferm30rep2.fastq 6172452

ferm0rep1.fastq 9043877

In [27]:
x = pd.DataFrame([
    [1, 0, 9043877, 94.25, 7291723,],
    [1, 30, 1721675, 96.14, 1402166, ],
    [2, 0, 9929568, 94.85, 7987001,],
    [2, 30, 6172452, 96.20, 4975466,]
])


x.columns = ['Replicate', 'Fermentation time', 'Total reads', '% aligned (HISAT2)', '# assigned (featureCounts)']
x.set_index(['Replicate', 'Fermentation time'])

Total reads  % aligned (HISAT2)  \
Replicate Fermentation time                                    
1         0                      9043877               94.25   
          30                     1721675               96.14   
2         0                      9929568               94.85   
          30                     6172452               96.20   

                             # assigned (featureCounts)  
Replicate Fermentation time                              
1         0                                     7291723  
          30                                    1402166  
2         0                                     7987001  
          30                                    4975466

In [30]:
pd.read_csv('gen_result_all.txt', sep='\t').dropna()

,Unnamed: 0,id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,gene-YER062C,gene-YER062C,16729.875894,7.899953,0.408404,19.343472,2.313608e-83,1.344669e-79
1,gene-YDR536W,gene-YDR536W,10088.387566,7.874257,0.411571,19.132209,1.361793e-81,3.957371e-78
2,gene-YHR094C,gene-YHR094C,8495.183817,7.881624,0.432049,18.242432,2.376454e-74,4.603984e-71
3,gene-YNL065W,gene-YNL065W,2109.592516,7.785356,0.466923,16.673751,2.034180e-62,2.955664e-59
4,gene-YKL120W,gene-YKL120W,1727.165761,7.269328,0.482440,15.067824,2.636549e-51,3.064725e-48
...,...,...,...,...,...,...,...,...
5807,gene-YBR156C,gene-YBR156C,83.490898,-0.003689,0.561237,-0.006573,9.947559e-01,9.954410e-01
5808,gene-YML038C,gene-YML038C,184.055055,0.002897,0.493135,0.005876,9.953120e-01,9.958260e-01
5809,gene-YHR187W,gene-YHR187W,304.151828,-0.001843,0.426352,-0.004322,9.965514e-01,9.968945e-01
5810,gene-YHR059W,gene-YHR059W,156.704164,0.000791,0.489437,0.001616,9.987104e-01,9.988823e-01


In [5]:
pd.read_csv('gen_result_50.txt', sep='\t')

,Unnamed: 0,id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,gene-YER062C,gene-YER062C,16729.875894,7.899953,0.408404,19.343472,2.313608e-83,1.344669e-79
1,gene-YDR536W,gene-YDR536W,10088.387566,7.874257,0.411571,19.132209,1.361793e-81,3.957371e-78
2,gene-YHR094C,gene-YHR094C,8495.183817,7.881624,0.432049,18.242432,2.376454e-74,4.603984e-71
3,gene-YNL065W,gene-YNL065W,2109.592516,7.785356,0.466923,16.673751,2.034180e-62,2.955664e-59
4,gene-YKL120W,gene-YKL120W,1727.165761,7.269328,0.482440,15.067824,2.636549e-51,3.064725e-48
5,gene-YLR264W,gene-YLR264W,1387.811877,6.483180,0.432160,15.001817,7.143608e-51,6.919775e-48
6,gene-YGR159C,gene-YGR159C,8021.263133,6.767420,0.451849,14.977169,1.035348e-50,7.521801e-48
7,gene-YJL122W,gene-YJL122W,1291.149172,6.511634,0.434559,14.984451,9.278892e-51,7.521801e-48
8,gene-YNL112W,gene-YNL112W,5328.977598,7.363983,0.493559,14.920173,2.436391e-50,1.573367e-47
9,gene-YOR271C,gene-YOR271C,1052.360380,6.844359,0.472759,14.477488,1.681312e-47,9.771787e-45


In [17]:
proc = subprocess.run('diff rna_genes.txt gen_genes.txt', shell=True, check=True, capture_output=True)
print(proc.stdout.decode('utf-8')) # no difference

In [10]:
for title in ('rna_genes.txt', 'gen_genes.txt'):
    with open(title, 'r') as f:
        genes = [line.strip('gene-').strip() for line in f.readlines()]
    with open('stripped_' + title, 'w') as f:
        f.write('\n'.join(genes))

# Gene ontology search results 

In [32]:
go_search = pd.read_csv('mapper_genes_15841_slimTab.txt', sep='\t')
go_search.head()

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
0,GO:0006364,rRNA processing,13,49,26.53%,366,6443,5.68%,"YDR449C, YEL026W, YER127W, YGR159C, YHR066W, Y..."
1,GO:0042273,ribosomal large subunit biogenesis,9,49,18.37%,130,6443,2.02%,"YCR072C, YDL063C, YEL026W, YHR066W, YIR012W, Y..."
2,GO:0042274,ribosomal small subunit biogenesis,8,49,16.33%,146,6443,2.27%,"YDR449C, YEL026W, YER127W, YGR159C, YHR196W, Y..."
3,GO:0042255,ribosome assembly,7,49,14.29%,79,6443,1.23%,"YCR072C, YGR159C, YHR066W, YIR012W, YLR264W, Y..."
4,GO:0006360,transcription by RNA polymerase I,6,49,12.24%,71,6443,1.10%,"YHR196W, YJL148W, YJR063W, YML043C, YMR093W, Y..."


In [34]:
genes = defaultdict(set)
for i, row in go_search.iterrows():
    term = row['TERM']
    term = term[0].upper() + term[1:]
    for gene in row['ANNOTATED_GENES'].split(', '):
        genes[gene].add(term)

In [71]:
deseq_top50 = pd.read_csv('gen_result_50.txt', sep='\t').drop(columns=['Unnamed: 0', 'stat', 'pvalue', 'baseMean'])

deseq_top50['id'] = deseq_top50['id'].str.strip('gene-')
deseq_top50['TERM'] = deseq_top50['id'].apply(lambda x: '; '.join(genes[x]))


upreg = deseq_top50.query('log2FoldChange > 0')
downreg = deseq_top50.query('log2FoldChange < 0')
print('UPREGULATED:')
display(HTML(upreg.to_html()))
print('DOWNREGULATED:')
display(HTML(downreg.to_html()))

UPREGULATED:


,id,log2FoldChange,lfcSE,padj,TERM
0,YER062C,7.899953,0.408404,1.344669e-79,Response to osmotic stress; Carbohydrate metabolic process
1,YDR536W,7.874257,0.411571,3.957371e-78,Carbohydrate transport; Transmembrane transport; Ion transport
2,YHR094C,7.881624,0.432049,4.603984e-71,Carbohydrate transport; Transmembrane transport; Ion transport
3,YNL065W,7.785356,0.466923,2.955664e-59,Amino acid transport; Response to chemical; Transmembrane transport; Ion transport
4,YKL120W,7.269328,0.482440,3.064725e-48,Transmembrane transport; Ion transport
5,YLR264W,6.483180,0.432160,6.919775e-48,Ribosomal small subunit biogenesis; Regulation of translation; Nucleobase-containing compound transport; RNA catabolic process; Cytoplasmic translation; RRNA processing; Ribosome assembly
6,YGR159C,6.767420,0.451849,7.521801e-48,Ribosomal small subunit biogenesis; DNA recombination; Ribosome assembly; RRNA processing
7,YJL122W,6.511634,0.434559,7.521801e-48,Ribosomal large subunit biogenesis
8,YNL112W,7.363983,0.493559,1.573367e-47,"Regulation of translation; Transcription by RNA polymerase II; RNA catabolic process; DNA-templated transcription, termination; RRNA processing"
9,YOR271C,6.844359,0.472759,9.771787e-45,Amino acid transport; Transmembrane transport; Ion transport


DOWNREGULATED:


,id,log2FoldChange,lfcSE,padj,TERM
28,YLR327C,-5.024976,0.392678,3.429409e-35,
46,YKR097W,-4.707023,0.392816,5.410520e-31,Carbohydrate metabolic process


Only 2 of the first 50 were downregulated, so let's check amongst all results...

In [53]:
deseq_all = pd.read_csv('gen_result_all.txt', sep='\t').drop(columns=['Unnamed: 0', 'stat', 'pvalue', 'baseMean'])

deseq_all['id'] = deseq_all['id'].str.strip('gene-')
deseq_all['TERM'] = deseq_all['id'].apply(lambda x: '; '.join(genes[x]))

# downreg_all = deseq_all.query('log2FoldChange < 0').query('TERM != ""')
# print('DOWNREGULATED:')
# display(HTML(downreg_all.head(100).to_html()))
downreg_all = deseq_all.query('log2FoldChange < 0')
downreg_all['id'].apply(lambda x: x.replace('LSR1', 'YGL181W')).head(100).to_csv('downreg_genes.txt', header=False, index=False)
display(HTML(downreg_all.head(100).to_html()))

,id,log2FoldChange,lfcSE,padj,TERM
28,YLR327C,-5.024976,0.392678,3.429409e-35,
46,YKR097W,-4.707023,0.392816,5.410520e-31,Carbohydrate metabolic process
69,YMR081C,-4.793440,0.428747,4.236803e-27,Cellular respiration
78,YDR342C,-4.506408,0.409684,2.819730e-26,Ion transport; Transmembrane transport; Carbohydrate transport
95,YCR021C,-4.450805,0.418637,1.285407e-24,Response to osmotic stress; Response to oxidative stress; Ion transport; Response to heat; Transmembrane transport; Cellular response to DNA damage stimulus; Response to chemical
96,YNL117W,-4.790761,0.451373,1.541315e-24,Monocarboxylic acid metabolic process; Carbohydrate metabolic process; Cellular respiration
100,YNR001C,-4.346953,0.412236,3.088563e-24,Carbohydrate metabolic process; Nucleobase-containing small molecule metabolic process; Cellular respiration
116,YBR169C,-4.141669,0.402047,3.447873e-23,Protein folding; Proteolysis involved in cellular protein catabolic process
121,YCL040W,-4.028336,0.395544,1.110131e-22,Carbohydrate transport; Transmembrane transport; Carbohydrate metabolic process; Nucleobase-containing small molecule metabolic process; Generation of precursor metabolites and energy; Monocarboxylic acid metabolic process
123,YJR073C,-4.008763,0.394980,1.565060e-22,Lipid metabolic process


In [49]:
go_search = pd.read_csv('downreg_annotation.txt', sep='\t')
go_search.head()
genes = defaultdict(set)
for i, row in go_search.iterrows():
    term = row['TERM']
    term = term[0].upper() + term[1:]
    for gene in row['ANNOTATED_GENES'].split(', '):
        genes[gene].add(term)